In [1]:
import random

from tqdm import tqdm

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv('all_info_tolstoy.csv', sep='@')  
df.sample(5)

,sentence,generated
1449,Взять револьвер и стрелять в него.—Но я не мог...,1
377,"- Вы, кажется, меня не узнаете?",0
716,"- Дай-то Бог, - проговорил он, - а то пора бы ...",0
1476,"- Альберт, опустив голову, смотрел на меня.–Я ...",1
225,- Он на ухо шепотом продолжал: - Я целовал ее ...,0


In [22]:
df = df.dropna() # удаляем получившиеся везде пустые значения

In [6]:
df1 = pd.read_csv('df1.csv', sep='@') 

In [23]:
df1 = df1.dropna()

In [7]:
df2 = pd.read_csv('df2.csv', sep='@') 

In [24]:
df2 = df2.dropna()

Подготавливаем данные для классификации

In [16]:
vectorizer = TfidfVectorizer()

In [25]:
# делим на предложения и целевую переменную
X = df.drop(['generated'], axis=1)['sentence']
y = df['generated']

In [27]:
# делим на сгенерированные и настоящие
generated = df1.drop(['generated'], axis=1)['sentence']
real = df2.drop(['generated'], axis=1)['sentence']

In [28]:
# векторизуем все
X_transformed = vectorizer.fit_transform(X)

In [29]:
# stratify -- для баланса классов в тестовой и трейновой
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3,
                                                    random_state=np.random.seed(42),
                                                    stratify=y)

In [30]:
# классификатор -- knn
classifier = KNeighborsClassifier()
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [31]:
y_pred_test = classifier.predict(X_test)

In [32]:
y_pred_train = classifier.predict(X_train)

In [33]:
acc_test = np.mean(y_test == y_pred_test)
acc_train = np.mean(y_train == y_pred_train)

Заодно проверим accuracy! На тесте совсем маленькая, но зато интереснее играть! :)

In [34]:
acc_test

0.553921568627451

In [35]:
acc_train

0.6712256706996318

С помощью сложных операций преобразования в numpy array и затем в list создаем супер-общую табличку с векторами каждого предложения, с самим предложением, с мнением модели и реальностью

In [36]:
X_tr = X_transformed.toarray().tolist()
X_tr_tr = X_train.toarray().tolist()
X_tr_ts = X_test.toarray().tolist()

In [37]:
new_arr = [0]*len(X)
for i in tqdm(range(len(X))):
    if X_tr[i] in X_tr_tr:
        new_arr[i] = y_pred_train[X_tr_tr.index(X_tr[i])]
    elif X_tr[i] in X_tr_ts:
        new_arr[i] = y_pred_test[X_tr_ts.index(X_tr[i])]

100%|██████████████████████████████████████████████████████████████████████████████| 2717/2717 [00:27<00:00, 99.69it/s]


In [38]:
new_df = pd.DataFrame(X_transformed.toarray())
new_df = new_df.assign(sentence=X)
new_df = new_df.assign(model=new_arr)
fin_df = new_df.assign(generated=y)  
fin_df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,6032,6033,6034,6035,6036,6037,6038,sentence,model,generated
123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,его задержал смотритель под предлогом неимения...,0,0.0
2311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"переносить ничего, а буду присылал",0,1.0
2129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"ночной рубашки живописно живописно ширмы, его ...",0,1.0
222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,- коротко и звучно отвечал Захар.,0,0.0
661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Я мучался, как мучаются 0,99 наших мальчиков.",0,0.0


In [39]:
np.mean(np.array(fin_df['model']) == np.array(fin_df['generated']))

0.6337872653662128

In [40]:
fin_df.to_csv('final_tolstoy.csv', index=False, sep='@')